In [1]:
import os
import re
import math
import requests
import psycopg2
import logging
import numpy as np
import pandas as pd

from time import sleep
from config import config
from random import randint
from bs4 import BeautifulSoup
from datetime import datetime
import psycopg2.extras as extras
from sqlalchemy import create_engine
from get_offers_by_city import connect

In [2]:
conn = connect(config())

Connecting to database...
Connection successful


In [8]:
query = '''
    SELECT * FROM all_offer_ids
'''

In [11]:
cursor = conn.cursor()
cursor.execute(query)

result = cursor.fetchall()
df = pd.DataFrame(result)
df.columns = ['extraction_datetime', 'offer_id', 'city', 'city_code', 'type']

In [13]:
df.shape

(6761, 5)

In [14]:
df.head()

,extraction_datetime,offer_id,city,city_code,type
0,2021-10-21 17:49:51,45695894,Dresden,100051,wohnung
1,2021-10-21 17:49:51,45649068,Dresden,100051,wohnung
2,2021-10-21 17:49:51,45311912,Dresden,100051,wohnung
3,2021-10-21 17:49:51,45690786,Dresden,100051,wohnung
4,2021-10-21 17:49:51,45693139,Dresden,100051,wohnung


In [30]:
def get_data_from_db(conn):
    '''Get the latest number os offers for each city.
    
    Parameters:
    ----------
        conn: connection to the database to extract the infos
        
    Return:
    -------
        Return a dataframe with the latest data about the number of offers in each city.
        
    '''   
    
    # create a cursor object
    cursor = conn.cursor()
    
    # get infos from the database
    query = '''
        SELECT * FROM all_offer_ids
    '''
    
    cursor.execute(query)
    result = cursor.fetchall()
    #df = pd.DataFrame(result)
    df_ids = pd.DataFrame(result, columns=['extraction_datetime', 'offer_id', 'city', 'city_code', 'type'])
    
    # get only the latest values
    #df = df.tail(11)
    #df.reset_index(inplace=True, drop=True)
    
    return df_ids

# get all infor for all founded offers
def get_offers_infos(df_ids, save=False):
    '''
    Here we will get the infos from the info panels for each offer.
    '''
    infos_list = []
    #count = 0
    now = datetime.now().strftime('%Y-%m-%d-%H:%M:%S')

    for Id in set(df_ids['offer_id']):

        url = f"https://www.immonet.de/angebot/{Id}?drop=sel&related=false&product=standard"
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
        page = requests.get(url, headers=headers)
        soup = BeautifulSoup(page.text, "html.parser")

        panels_infos = soup.findAll('div', class_='row box-50')
        # get all infos about each offer (all infos are mixed)

        offer_infos = []
        for panel in panels_infos:
            text = panel.text.replace('\n', '').replace('\t', '-')
            offer_infos.append(text)



        infos_list.append({'offer_id': Id,
                           'extraction_date': now,
                           'city': df_ids['city'][count],
                           'city_code': df_ids['city_code'][count],
                           'offer_type': df_ids['type'][count],
                           'offer_infos': offer_infos})
        sleep(1)
        #print(count)
        #count += 1

    df_infos = pd.DataFrame(infos_list)
    df_infos.drop_duplicates(subset='offer_id', inplace=True)
    
    if save:
        now2 = datetime.now().strftime('%Y_%m_%d')
        df_infos.to_csv(f'../data/df_infos_{now2}.csv', index=False)
        
    return df_infos

In [19]:
df1 = get_data_from_db(conn)

In [22]:
df_ids = df1

In [20]:
df1.head()

,extraction_datetime,offer_id,city,city_code,type
0,2021-10-21 17:49:51,45695894,Dresden,100051,wohnung
1,2021-10-21 17:49:51,45649068,Dresden,100051,wohnung
2,2021-10-21 17:49:51,45311912,Dresden,100051,wohnung
3,2021-10-21 17:49:51,45690786,Dresden,100051,wohnung
4,2021-10-21 17:49:51,45693139,Dresden,100051,wohnung


In [ ]:
# get all infor for all founded offers
def get_offers_infos(df_ids, save=False):
    '''
    Here we will get the infos from the info panels for each offer.
    '''
    count = 0
    #infos_list = []
    #datetime_list = []
    now = datetime.now().strftime('%Y-%m-%d-%H:%M:%S')

    for Id in set(df_ids['offer_id']):

        url = f"https://www.immonet.de/angebot/{Id}?drop=sel&related=false&product=standard"
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
        page = requests.get(url, headers=headers)
        soup = BeautifulSoup(page.text, "html.parser")

        panels_infos = soup.findAll('div', class_='row box-50')
        # get all infos about each offer (all infos are mixed)

        offer_infos = []
        for panel in panels_infos:
            text = panel.text.replace('\n', '').replace('\t', '-')
            offer_infos.append(text)



        infos_list.append({'offer_id': Id,
                           'extraction_date': now,
                           'city': df_ids['city'][count],
                           'city_code': df_ids['city_code'][count],
                           'offer_type': df_ids['type'][count],
                           'offer_infos': offer_infos})
        #print(count)
        count += 1

        #if len(offer_infos) != 0:
        #    datetime_list.append(datetime.now().strftime('%Y-%m-%d-%H:%M:%S'))
        #    infos_list.append(offer_infos)
        #else:
        #    infos_list.append(np.nan)
        #    print('ERROR')

    df_infos = pd.DataFrame(infos_list)
    df_infos.drop_duplicates(subset='offer_id', inplace=True)
    
    if save:
        now2 = datetime.now().strftime('%Y_%m_%d')
        df_infos.to_csv(f'../data/df_infos_{now2}.csv', index=False)
        
    return df_infos